In [7]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd

data = pd.read_excel('/content/Train.xlsx')
data

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,240000,2,2,2,37,-2,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
1,240000,2,3,1,43,0,0,0,0,0,...,65130,66494,67441,2300,2400,2329,2411,2050,2000,0
2,40000,2,1,2,24,0,0,0,0,0,...,37263,35224,34111,5000,8017,7000,4000,5000,5000,1
3,360000,1,1,2,29,-2,-2,-2,-2,-2,...,3466,1630,2931,2358,3788,3483,1638,2946,16950,0
4,130000,1,1,2,27,0,0,0,0,0,...,35194,35951,36703,1867,1872,1256,1500,1500,1500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23994,80000,2,2,2,26,1,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
23995,50000,2,3,2,28,1,3,3,2,2,...,40165,42683,43843,2500,0,0,3200,2000,1700,0
23996,500000,1,1,2,33,-1,-1,-1,-2,-1,...,4158,7958,-196,2065,10556,4170,7963,196,7083,0
23997,50000,1,2,2,42,2,2,2,2,-1,...,22402,9528,8447,0,3701,554,10026,421,6106,0


In [8]:
import autosklearn
print('autosklearn: %s' % autosklearn.__version__)

autosklearn: 0.15.0


In [36]:

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autosklearn.classification import AutoSklearnClassifier
# define dataset
X, y = make_classification(n_samples=100, n_features=15, n_informative=5, n_redundant=5, random_state=1)
# split into train and test sets
X_Train, X_Test, y_Train, y_Test = train_test_split(data.drop(columns="default.payment.next.month"), data["default.payment.next.month"], test_size=0.2, random_state=7)
# define search
model = AutoSklearnClassifier(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=-1, ensemble_kwargs = {'ensemble_size': 15},resampling_strategy_arguments={
    "train_size": 0.7,     # The size of the training set
    "shuffle": True,        # Whether to shuffle before splitting data
    "folds": 10,           # Used in 'cv' based resampling strategies
    "methods": ["precision", "subsample"]
} )

# perform the search
model.fit(X_Train, y_Train)
# summarize
print(model.sprint_statistics())
# evaluate best model
y_hat = model.predict(X_Test)
acc = accuracy_score(y_Test, y_hat)
print("Accuracy: %.3f" % acc)

auto-sklearn results:
  Dataset name: 588ec19b-c210-11ed-81b3-0242ac1c000c
  Metric: accuracy
  Best validation score: 0.823438
  Number of target algorithm runs: 24
  Number of successful target algorithm runs: 7
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 17
  Number of target algorithms that exceeded the memory limit: 0

Accuracy: 0.820


In [30]:
pprint(model.show_models(), indent=4)


NameError: ignored

In [32]:
print(model.leaderboard())


          rank  ensemble_weight               type      cost   duration
model_id                                                               
7            1              0.6  gradient_boosting  0.177557   8.168239
6            2              0.2  gradient_boosting  0.182134   8.554486
8            3              0.2  gradient_boosting  0.207702  10.144601


In [34]:
model.show_models()

{6: {'model_id': 6, 'rank': 1, 'cost': 0.18213383838383834, 'ensemble_weight': 0.2, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f455b2cbd00>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f45838cdbb0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f455f871580>, 'sklearn_classifier': HistGradientBoostingClassifier(early_stopping=True,
                               l2_regularization=2.0357979007621058e-10,
                               learning_rate=0.04202246787451397, max_iter=512,
                               min_samples_leaf=14, random_state=1,
                               validation_fraction=None, warm_start=True)}, 7: {'model_id': 7, 'rank': 2, 'cost': 0.17755681818181823, 'ensemble_weight': 0.6, 'data_preprocessor': <autosklearn.pipeline.components.data_pr